In [259]:
# 8A
import os
#os.chdir('/users/justinehly/github/advent_of_code/day 8')
os.chdir(r'C:\Users\justi\GitHub\advent_of_code\day 8')


#with open('example.txt') as f:
#    lines = f.readlines()

with open('input.txt') as f:
    lines = f.readlines()

input = [i.split('|')[1] for i in lines] # grab just the data after the pipe delimiter and remove the key return
input = [l.split() for l in input] # divide each of the outputs into its own list element
cipher = [i.split('|')[0] for i in lines] # grab cipher before the pipe
cipher = [i.split() for i in cipher] # divide each of the codes into individual strings and remove the return marker

unique_nums = [2, 4, 3, 7]  # length of the signals we are targeting

len([x for i in input for x in i if len(x) in unique_nums]) # count the signals that match the lengths we are looking for 


554

In [260]:
import copy
def split(word):
    return [letter for letter in word]

def one(cipher):
    return ["".join(i) for i in cipher if len(i) == 2][0]

def four(cipher):
    return ["".join(i) for i in cipher if len(i) == 4][0]

def seven(cipher):
    return ["".join(i) for i in cipher if len(i) == 3][0]

def eight(cipher):
    return ["".join(i) for i in cipher if len(i) == 7][0] 

def zero_six_nine(cipher):
    return ["".join(i) for i in cipher if len(i) == 6] 

def two_3_5(cipher):
    return ["".join(i) for i in cipher if len(i) == 5] 

def number_cipher(cipher):
    numbers = {}  # keep track of the numbers as we discover them
    code = {} # keep track of how to map the letters based on the 7 positions of the analog read_out
    '''
                 top
                 ----               |
    left_upper  |    |  right_upper |
                |    |              |
        middle   ----               | right
    left_lower  |    | right_lower  |
                |    |              | 
                 ----               |
                bottom
    '''
    
    # pull out the #8
    numbers[8] = eight(cipher)

    # find the right pair of letters using #1
    numbers[1] = one(cipher)
    right = split(numbers[1])
    #print('right: %s' % right)
    
    # find the top of the cipher using #7
    numbers[7] = seven(cipher)
    l7 = split(numbers[7])
    code['top'] = [x for x in l7 + right if x not in l7 or x not in right] # only return the letter from seven that is not on the right
    code['top'] = code['top'][0] # extract the letter from the list
    #print('top: %s' % code['top'])


    # find middle and upper_left using #4
    numbers[4] = four(cipher)
    l4 = split(numbers[4])
    upper_left_middle = [i for i in l4 + right if i not in l4 or i not in right]  # remove the right labels, these values are either the upper_left or the middle
    #print('up_left-middle: %s' % upper_left_middle)

    # - - - 0 6 9 - - - 

    # these are either 0, 6, 9 
    l069 = zero_six_nine(cipher)
    l069 = [split(x) for x in l069]
    #print(l069)

    # these are either 0, 6, 9, but the 0 won't include the middle letter while the 6/9 will share that value
    temp = [[x for x in i if x in right] for i in l069] # determine which of these codes includes both of the right letters
    lower_right = [x for x in temp if len(x) == 1][0]  # the number six will only have 1 right letter (the lower one)
    
    # separate uppoer and lower right letters
    code['lower_right'] = lower_right[0] # remove letter from list
    code['upper_right'] = [x for x in right if x not in lower_right][0]
    
    
    # isolate the #6
    idx_6 = temp.index(lower_right) # determine the position of six 
    numbers[6] = "".join(l069[idx_6]) #this is the #6
    
    # isolate the upper left letter
    temp = [[x for x in i if x in upper_left_middle] for i in l069] # determine which of these codes includes both of the upper_left and the middle letters
    upper_left = [x for x in temp if len(x) == 1][0] # the zero will only have the upper_left letter
    
    # separate the upper_left and middle letters
    code['upper_left']  = upper_left[0]
    code['middle'] = [x for x in upper_left_middle if x not in upper_left][0]
    
    # isolate the zero
    idx_0 = temp.index(upper_left) #determine position of the zero
    numbers[0] = "".join(l069[idx_0])
    
    # get the #9
    idx_list = [x for x in range(len(l069))]
    idx_list.remove(idx_6)
    idx_list.remove(idx_0)
    idx_list[0]
    numbers[9] = "".join(l069[idx_list[0]])
    

    # - - - 2 3 5 - - - 

    # these numbers are either 2,3,5
    l235 = two_3_5(cipher)
    l235 = [split(x) for x in l235]
    #print('2-3-5: %s' % l235)
    
    # 5 will be the only one of these values that contains the upper_left letter
    temp = [[x for x in i if x in upper_left] for i in l235] # deteremine which of these codes contains the upper left letter
    temp = [1 if x else 0 for x in temp] # use binary markers to find the non empty list (the number 5)
    idx_5 = temp.index(1)
    numbers[5] = "".join(l235[idx_5])
    #print(temp, idx_5)
    
    # find the bottom since we know all the other letters in the number 5
    find_bottom =  [code['top'], code['middle'], code['upper_left'], code['lower_right']]
    code['bottom'] = [x for x in split(numbers[5]) if x not in find_bottom]
    code['bottom'] = code['bottom'][0]
    find_ll = find_bottom
    find_ll.append(code['bottom'])
    find_ll.append(code['upper_right'])
    
    code['lower_left'] = [x for x in split(numbers[8]) if x not in find_ll]
    code['lower_left'] = code['lower_left'][0]
    
    numbers[2] = "".join([code['top'], code['upper_right'], code['middle'], code['lower_left'], code['bottom']])
    numbers[3] = "".join([code['top'], code['upper_right'], code['middle'], code['lower_right'], code['bottom']])
    
    # sort of the codes for each number
    numbers = {k: sorted(v) for k,v in numbers.items()}
    numbers = {k: "".join(v) for k,v in numbers.items()}
    
    
    #print(numbers)
    #print(code)
    return numbers, code

def dash_display(codes):
    return ["".join(sorted(i)) for i in codes]

def match(numbers, codes):
    output = []
    for code in codes:
        output.append([key for key,value in numbers.items() if value == code])
        
    return output

def format_output(output):  
    output = [o[0] for o in output]
    output = [str(o) for o in output]
    output = int("".join(output))
    return output

def results(cipher, input):
    numbers, code = number_cipher(cipher)
    codes = dash_display(input)
    init_output = match(numbers, codes)
    output = format_output(init_output)
    return output

def fix_display(cipher, input):
    display = []
    for n, codes in enumerate(cipher):
        display.append(results(codes, input[n]))
    return sum(display)

fix_display(cipher, input)